# Exercicios

Utilize o dataset train.csv para os exercicios.

#### Estamos em 2015 e você trabalha em uma empresa que compra filmes para passar no cinema. Bons filmes custam caro, assim como filmes ruins barato, mas Bons filmes atraem mais público. Seu chefe sabe que você está aprendendo Machine Learning e te propõe a seguinte tarefa:  - Precisamos achar uma boa oportunidade que nos faça aumentar o lucro da empresa!

Você conseguirá aumentar o lucro da empresa quando comprar um filme que é subvalorizado mas que as features dele indicam que será um bom filme.

Se pudessemos descobrir quais filmes que as pessoas acham que seriam ruins mas na verdade são bons, poderiamos investir apenas nestes e ganhar a diferença já que foi mal precificado.


Vamos partir dos seguintes pressupostos:
- Um filme bom ou ruim do ponto de vista de quem está precificando será o tamanho do seu orçamento (budget).

- Um filme Bom/Ruim do ponto de vista do público serão filmes com altas notas (imdb_score).

Então vamos procurar os filmes que tenham uma boa relação entre budget e score.
(é como se quisessemos comprar scores pois isso que o público quer ver mas os donos dos filmes vendem baseado no budget do filme).

O primeiro passo é entendermos bem o problema e os dados disponíveis.

Por exemplo, se vamos comprar um filme, algumas features não estarão disponiveis antes da compra (elas são criadas apenas após a compra) para predizer o rating como as vendas de ingresso, faturamento, etc.

Depois separamos nos nossos dados quais serão nossas variáveis explicativas (Os X) e qual é nossa variável target (nosso Y).

Caso algumas das nossas variáveis explicativas sejam categórias, precisamos transforma-las em numéricas para que o modelo consiga entende-la.

Podemos utilizar uma função do pandas dessa forma:

``` df = pd.get_dummies(df) ```

O último passo antes de rodar um modelo é dividir nossos dados em duas partes, uma para treinar o modelo e outra para testa-lo, podemos fazer uma amostra aleatória ou usar a função do sklearn que nos ajuda com isso:

```
from sklearn.model_selection import train_test_split

 X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.33, random_state=42)
```
Para isso Treine um Modelo de Machine Learning supervisionado que aprenda a predizer as notas dos filmes.

Utilize os 4 passos visto na primeira aula de machine learning:
```from sklearn.linear_model import LinearRegression```

Após treinar o modelo vamos ver como estão as [métricas](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) de assertividade.

Analisar o R2, RMSE, MSE. Qual é o melhor nesse caso?

Exemplo do MSE
```
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true, y_pred)
```

Como poderiamos melhorar esse modelo? Será que existe algum método para colocar todas as variáveis ao quadrado, cubo, etc para que possamos capturar efeitos não lineares? Óbviamente existe e já foi implementado no sklearn, podemos usar a função:

```
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)

df = poly.fit_transform(df)
```

Aparentemente sempre que aumentamos o grau do polinomio nosso modelo fica melhor, porque não podemos botar um número alto como 10?

Rode o algoritmo com um grau = 10 e compare as métricas de treino contra as métricas de teste. O que está acontecendo?

Parece que estamos chegando perto de um modelo bom para nosso objetivo de identificar oportunidade nos filmes. Será que poderiamos utilizar outro método de regressão mais robusto? Vamos tentar utilizar um dos melhores métodos que utiliza arvores aleatórias (usado geralmente em classificação e as estudaremos mais profundamente no módulo 4) como modelo:

Treine agora o mesmo modelo utilizando um modelo não-linear (não exige que as correlações sejam lineares) e compare as métricas.

```from sklearn.ensemble import RandomForestRegressor```



Agora estamos em 2016. Use seu modelo treinado com todos os filmes até 2015 para responder a pergunta do seu chefe com os filmes que apareceram em 2016: Quais são os filmes que são realmente oportunidades de ganhar dinheiro (altos ratings, baixo custo).

# Passo 1 - Entender o problema

O problema está descrito no tutorial acima. O que não está escrito é qual abordagem deveriamos ter para resolve-lo.

Vamos pensar em uma possibilidade de solução aqui e escreveremos todo o restante do código pensando em como alcançar esse resultado.

Não existe um só caminho para resolver esse problema, mas a solução que eu pensei em aplicar é a seguinte:

    1) de um lado os donos dos filmes precificam seus produtos com base em quanto gastaram.

    2) Seguindo o pressuposto do problema, o sucesso de bilheteria do filme depende apenas das notas que as pessoas vão dar a ele.

Com isso chegamos a conclusão que o melhor filme para apostarmos são aqueles que a relação entre o preço (1-custo) e o (2-receita) é a mais vantajoso para nós.

Uma forma de pensar é que temos que "comprar" scores então procuraremos os scores mais baratos que será dado por (1)/(2) -> custo do filme / beneficio do filme.

Então precisamos treinar um modelo de machine learning que quando chegar um filme novo, prediga com o máximo de acurácia a nota média que as pessoas darão para ele ( o quociente na nossa equação) já que o númerador (o custo do filme) já saberemos.

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_raw = pd.read_csv('train.csv')

In [4]:
# agora vamos fazer uma cópia do nosso dataframe.
# Note que usamos a função deep=True pois só assim garantimos que o 
# dado está duplicado e não é apenas um "atalho" para o dataset original
df = df_raw.copy(deep=True)

# Passo 2 - Escolher as variáveis.


Definido claramente o que vamos fazer, vamos olhar as variáveis disponiveis e ver o que faz ou não sentido incluir.

Diferentemente de modelagem estatistica, em machine learning não estamos preocupados com as variáveis serem correlacionadas, então não há um bom motivo (por hora) para retirarmos variáveis do modelo, portanto, meu primeiro modelo de 'benchmark' terá o máximo de variáveis/features possível.

Algumas são facilmente excluidas, pois mesmo que sejam boas preditoras como o tamanho da bilheteria, ela só estará disponível após o filme ser lançado. Parte bastante importante da definição de um modelo de machine learning é pensar nesses pontos, de quais variáveis estarão disponiveis em um modelo de produção e isso, muitas vezes, pode ser bastante complicado por conta do carater temporal das variáveis tornando-se um "leakage" ou seja, um vazamento de dados.


Variáveis como gross sabemos que não estará disponivel antes do lançamento então não vamos usa-la para treinar nosso modelo.

Outra variável que não faz sentido para um modelo de machine é aquela que identifica o filme como um id, index ou o nome do filme.

In [13]:
list(df_raw)

['Unnamed: 0',
 'color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'gross',
 'genres',
 'actor_1_name',
 'movie_title',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'plot_keywords',
 'movie_imdb_link',
 'num_user_for_reviews',
 'language',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio',
 'movie_facebook_likes']

In [14]:
columns_to_drop = ['Unnamed: 0', 'gross', 'movie_title', 'plot_keywords']

In [15]:
df = df_raw.drop(columns_to_drop, axis=1)

In [16]:
list(df)

['color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'genres',
 'actor_1_name',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'movie_imdb_link',
 'num_user_for_reviews',
 'language',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio',
 'movie_facebook_likes']

# Passo 3 - Tratando as variáveis.

**Essa é a etapa mais longa, mais importante e mais dificil de um problema de machine learning.** (falaremos mais sobre essas técnicas nas próximas aulas)

Essa etapa podemos usar todos os tratamentos de variáveis que aprendemos na modelagem estatistica quando estavamos rodando regressões simples. Não vou repetir os tratamentos mas todos podem ser utilizados.


#### Campos nulos.

Quando rodamos um dff.describe() percebemos muitos campos nulos e há diversas formas de trata-los.

Alternativas para tratar nulos.

1) Deleter todos os nulos.

```
df = df.dropna()
```

2) Substituir os nans pelas médias (nas numéricas) ou modas (nas categóricas).

df['numericas'] = df['numericas'].fillna(df.mean())


3) rodar um "pré-ML" que prevê os valores faltantes.



4) Usar um algoritmo robusto para campos nulos.

ex:

```
import xgboost as xgb
```

Vamos usar uma abordagem mista. Nas variáveis numéricas vamos substituir pela média e nas variáveis categóricas vamos excluir as linhas com campos nulos para aprender as duas metodologias. 

(conforme vimos em estatistica, excluir linhas que não são aleatórias enviesam nosso modelo e não deve ser feito a menos que você tenha certeza que aqueles nulos são por alguma falha aleatória o que é bastante raro. Não sendo aleatório devemos tratar com uma das outras metodologias, principalmente a número 4).

In [17]:
df.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
genres                        object
actor_1_name                  object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
movie_facebook_likes         float64
dtype: object

In [18]:
df_numeric = df.select_dtypes(include=[np.number]) # dataset apenas com colunas numéricas.
numericas = list(df_numeric)

In [19]:
# Nessa função selecionamos todas as colunas númericas e substituimos seus 
# valores pelas médias das colunas. A função fillna retorna todos os campos 
# nulos e a função df.mean retorna as médias de cada coluna.
df[numericas] = df[numericas].fillna(df.mean())

In [20]:
# já nas variáveis categóricas (que contém texto ao invés de números)
# vamos substituir os campos nulos por uma nova string que representa que aquele campo é faltante.
df = df.fillna('na') # substitui os dados faltantes por 'na'
df = df.dropna() # se sobrar alguma linha (nao deveria), dropamos essas linhas.
df.shape

(4937, 25)

In [21]:
df.shape

(4937, 25)

Nossa segunda etapa de tratamento das variáveis consiste em transformar as variáveis em funções lineares (se estivermos rodando modelos lineares) através da aplicação de log nas variáveis "explosivas" tornando-a linear. Não faremos essa etapa pois nosso objetivo é rodar um algoritmo não linear em seguida.

A terceira etapa de preparação dos dados é criar variáveis dummies para nossas features categóricas.

Basicamente os algoritmos de machine learning não conseguem interpretar textos como o genêro de um filme ou os atores que participaram então precisamos transformar essas colunas categóricas em colunas numéricas para que o algoritmo possa capturar seus efeitos e predizer nosso target.

Substituir as categorias por números dessa forma: COluna genêro de filme com valores [Terror=1, Comédia=2, Ação=3] embora faça o algoritmo rodar, não funcionará bem como variável explicativa pois elas não representam uma sequencia numérica. A forma mais simples de fazermos isso virar uma variável explicativa é pegar cada uma das categorias possíveis e transformar em uma coluna forma única que recebe apenas dois valores, 0 e 1. 0 se o valor não estiver presente e 1 caso esteja presente. Dessa forma nossa  representação ficará da seguinte forma. 

- Coluna 1 = genero_terror
- Coluna 2 = genero_comédia
- Coluna 3 = genero_ação

Assumindo 1 ou 0 dependendo do filme.

Uma observação sobre essa técnica é que não é necessário colocar todas as possibilidades, uma delas se torna redundante e ao modelar esse caso podemos excluir uma delas.

Essa técninca chamamos de one_hot (sklearn) e no pandas temos a função get_dummies que faz o mesmo processo.

In [22]:
df = pd.get_dummies(df, drop_first=True)

In [23]:
df.shape

(4937, 16728)

Aumentamos das 26 colunas originais para mais de 15mil colunas!

# Passo 4 - Treinando o modelo de Machine Learning

Essa é a etapa mais simples mas que exige mais experiência.

1) qual algoritmos escolher?

Sabemos que nosso problema se trata de prever uma variável continua (uma nota) e como temos diversas notas para treina-lo podemos usar uma abordagem supervisionado.

(obs: poderiamos tentar transformar cada intervalo de notas em uma categoria, ex: [0,1] -> 1, [1,2] -> 2 .... [9,10] -> 10 e prever usando um algoritmo de classificação, mas como as notas fazem sentido como uma variável numérica, ou seja, a nota seguinte é a nota anterior + 1, um algoritmo de classificação dificilmente superaria um de regressão já que as probabilidades de cada classe prevista em uma classificação é independente, é como se uma nota não tivesse nenhuma relação com a outra e isso não é verdade no nosso caso).

Então:

    Supervisionado -> Regressão
    
    
Mas há dezenas de algoritmos de regressão (ver todos os disponiveis no sklearn nesse link: http://scikit-learn.org/stable/supervised_learning.html )

Alguns deles:
- MQO (o mais simples que estudamos em estatistica e abrimos o código na aula2 de machine learning).
- Ridge -> Método de Shrinkage. Conrola sobreajuste com L1

    $L = ∑( Ŷi– Yi)^2 + λ∑ β^2$


- lasso -> Também método de Shrinkage. controle sobreajuste com L2

   $ L = ∑( Ŷi– Yi)^2 + λ∑ |β|$


- elasticnet -> combinação de Lasso + Ridge
- Regressão Bayesiana -> Retorna distribuições de probabilidade ao invés de simples valores.

    Ex: https://towardsdatascience.com/introduction-to-bayesian-linear-regression-e66e60791ea7?gi=392eaf9ea3b 
    
- Arvores aleatórias -> insere não lineariedades
- Redes neurais -> também insere não lineariedades.
- Ensambles -> Combinação de diferentes modelos.
- Etc


Qual usar ?

No nosso exemplo vamos usar o mais simples MQO para usar de comparativo e um extremamente poderoso que são as arvores aleatórias (que ainda não vimos seu funcionamento).

Na prática, quando colocamos um modelo em produção, podemos rodar diferentes algoritmos (que tem suas próprias vantagens e desvantagens) e pegar uma combinação desses diferentes algoritmos como o resultado final. A essa combinação de modelos damos o nome de **ensamble**.

Para rodar qualquer modelo de machine learning desses, vamos passar pelos mesmos passos:

    1) Importar o modelo desejado.
    2) Instanciar em uma variável com os parâmetros desejados (ainda não vimos os parâmetros).
    3) Separar nossos dados em variáveis explicativas (X) e explicadas/target (Y)
    4) Separar nossos dados em Treino e Teste
    5) Treinar o Modelo com o .fit()
    6) Analisar as métricas, se não estiver boas, voltamos aos passos anteriores de trabalhar com as variáveis.
    7) Estando tudo ok, rodamos previsões

**1) Importar modelos**

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

**2) Instanciar os modelos em variáveis**

In [30]:
modelo_MQO = LinearRegression()
modelo_RF = RandomForestRegressor()

**3) Separar os dados em X e Y**

In [31]:
X = df.drop(['imdb_score'], axis = 1)
Y = df['imdb_score']

**4) Separar os dados em Treino e Teste ** 

Esse ponto é crucial em machine learning e falaremos mais vezes sobre diferentes técnicas em "splitar" os dados.

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [33]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3949, 16727)
(988, 16727)
(3949,)
(988,)


In [34]:
900 / (900 + 3600)

0.2

**5) Treinar o Modelo para que o algoritmo descrubra os melhores betas (que produzem os menores erros)**

In [35]:
modelo_MQO.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**6) Após treinar o modelo podemos analisar suas métricas e aqui é uma parte sensivel, vamos entender algumas métricas.**

In [36]:
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error

yhat_train = modelo_MQO.predict(X_train) # previsao dos dados de treino para calcular as métricas
yhat_test = modelo_MQO.predict(X_test) # previsao dos dados de teste para calcular métricas

print('-----Dados de Treino-----')
print('MSE - treino', mean_squared_error(y_train, yhat_train))
print('MAE - treino', median_absolute_error(y_train, yhat_train))
print('R2 - treino', r2_score(y_train, yhat_train))

print('\n-----Dados de Teste-----')
print('MSE - test', mean_squared_error(y_test, yhat_test))
print('MAE - test', median_absolute_error(y_test, yhat_test))
print('R2 - test', r2_score(y_test, yhat_test))

-----Dados de Treino-----
MSE - treino 3.4614227492080305e-10
MAE - treino 1.160846905179369e-05
R2 - treino 0.9999999997291479

-----Dados de Teste-----
MSE - test 0.6794077192742255
MAE - test 0.47469819667708535
R2 - test 0.4369571865956374


Vamos entender primeiro as métricas e em seguida entender a diferença entre os números de treino e de teste.

**O que é o MSE e o MAE**
MSE: Erros médios quadraticos.
MAE: Erros médios absolutos.

Qual a diferença entre eles?

Pensemos no caso de um dataset com muitos outlinears, qual das duas métricas será mais prejudica (ou seja, qual métrica é mais sensivel a dados muito fora do padrão?). Nosso métrica ao quadrado é mais sensivel pois pegamos a diferença entre o valor predito e o realizado e elevamos ao quadrado tornando a diferença ainda maior. Já o MAE não é tão sensivel aos valores extremos pois consideramos apenas a distância absoluta que da o mesmo peso a distâncias grandes e pequenas.
Ex: Valor predito = 8, Valor Real = 5.
MAE= 8-5 = 3
MSE= (8-5)^2 = 9 -> Penaliza erros maiores.


Já o R2 é a mesma intepretação que demos em modelagem estatistica (% da variação de Y explicada por todas as variáveis explicativas X). Só que agora nosso objetivo é aumentar ao máximo nosso R2.



** Métricas de Treino e Métricas de Teste **

O grande objetivo de um modelo de aprendizado de maquina (ML) é termos um modelo que é preditivo para novos dados, ou seja, precisamos de um modelo que seja bastante genérico, que depois de treinado pode pegar dados que não estavam na amostra e consiga prever seu target com uma boa precisão.

Então ter boas métricas nos dados de treino não significam necessariamente que o modelo é um bom preditor. Para isso ele precisa ter boas métricas nos dados de teste! (aquela parte dos dados que separamos e não participaram do treino). Se nosso algoritmo é capaz de prever de maneira satisfatória nossos dados de teste (que são dados novos para ele já que não participaram do treino) esse é um bom modelo.

Quando as métricas dos dados de treino estão muito superiores aos dados de teste (como no nosso exemplo) significa que estamos fazendo um superajuste/sobreajuste ou como gostamos de chamar o modelo está com overfiting. Significa basicamente que nosso modelo se tornou especialista em prever nossos dados de treino mas não necessariamente é um bom modelo para prever dados fora do treinamento. Há muitas formas de contornar isso que veremos nas próximas aulas, mas a mais importante que faremos aqui é tentar equilibrar a complexidade do modelo, basicamente colocamos mais variáveis do que seria necessario.

É aqui que voltamos ao passo de escolher e tratar variaveis e rodamos tudo novamente.
Vamos fazer todos os processos na próxima linha e colocar um modelo um pouco mais simples:


<img src="img\overfit.png" style="height:250px">

In [21]:
# Vamos remover algumas colunas que podem estar causando um superajuste/overfiting no nosso modelo

In [22]:
list(df_raw)

['Unnamed: 0',
 'color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'gross',
 'genres',
 'actor_1_name',
 'movie_title',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'plot_keywords',
 'movie_imdb_link',
 'num_user_for_reviews',
 'language',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio',
 'movie_facebook_likes']

In [23]:
for column in list(df_raw):
    if 'likes' in column:
        pass
    elif 'name' in column or 'link' in column or 'Unnamed: 0' in column or 'plot_keywords' in column:
        print(column)

Unnamed: 0
director_name
actor_2_name
actor_1_name
actor_3_name
plot_keywords
movie_imdb_link


In [ ]:
overfiting_columns = []

for column in list(df):
    if 'likes' in column:
        pass
    elif 'name' in column or 'link' in column or 'Unnamed: 0' in column:
        overfiting_columns.append(column)
        
print(f'#columns_to_drop: {len(overfiting_columns)}')
df = df.drop(overfiting_columns, axis=1)
df = pd.get_dummies(df)

print(f'new_shape: {df.shape}')

In [ ]:
def run_mqo(df, model):
    X = df.drop(['imdb_score'], axis = 1)
    Y = df['imdb_score']

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)    
    model.fit(X_train,y_train)

    yhat_train = model.predict(X_train) # previsao dos dados de treino para calcular as métricas
    yhat_test = model.predict(X_test[list(X_train)]) # previsao dos dados de teste para calcular métricas

    print('-----Dados de Treino-----')
    print('MSE - treino', mean_squared_error(y_train, yhat_train))
    print('MAE - treino', median_absolute_error(y_train, yhat_train))
    print('R2 - treino', r2_score(y_train, yhat_train))

    print('\n-----Dados de Teste-----')
    print('MSE - test', mean_squared_error(y_test, yhat_test))
    print('MAE - test', median_absolute_error(y_test, yhat_test))
    print('R2 - test', r2_score(y_test, yhat_test))
    
run_mqo(df, LinearRegression())

Vamos passar agora agora um segundo modelo que consegue capturar, entre outras coisas, não lineariedades:

In [26]:
run_mqo(df, RandomForestRegressor())

-----Dados de Treino-----
MSE - treino 0.12606231766612644
MAE - treino 0.16000000000000014
R2 - treino 0.8985825079310189

-----Dados de Teste-----
MSE - test 0.7349672874493928
MAE - test 0.4399999999999995
R2 - test 0.4454201027866417


Parece bem melhor agora, tanto o R2 aumentou como ficou mais parecido entre treino e teste sugerindo que não estou dando overfiting no modelo.

Podemos ainda criar modelos um pouco mais complexos inserindo formas quadraticas, cubicas, etc para tentar capturar algum efeito não linear nos dados.

Idealmente plotariamos as variáveis e escolheriamos as que fazem mais sentido colocar variaveis quadraticas, etc. Mas na prática rodamos um algoritmo para passar um polinomio em todas as nossas variáveis como no código abaixo:

Obs: Colocamos todos os passos da regressão em uma função para não termos que ficar repetindo mais códigos.

In [27]:
def print_metrics(y_train, yhat_train,y_test, yhat_test):
    print('\n-----Dados de Treino-----')
    print('MSE - treino', mean_squared_error(y_train, yhat_train))
    print('MAE - treino', median_absolute_error(y_train, yhat_train))
    print('R2 - treino', r2_score(y_train, yhat_train))

    print('\n-----Dados de Teste-----')
    print('MSE - test', mean_squared_error(y_test, yhat_test))
    print('MAE - test', median_absolute_error(y_test, yhat_test))
    print('R2 - test', r2_score(y_test, yhat_test))

In [28]:
from sklearn.preprocessing import PolynomialFeatures

numeric_columns = ['num_critic_for_reviews',
                     'duration',
                     'director_facebook_likes',
                     'actor_3_facebook_likes',
                     'actor_1_facebook_likes',
                     'num_voted_users',
                     'cast_total_facebook_likes',
                     'facenumber_in_poster',
                     'num_user_for_reviews',
                     'budget',
                     'title_year',
                     'actor_2_facebook_likes',
                     'aspect_ratio',
                     'movie_facebook_likes']

dummies = []

for column in list(df):
    if 'imdb_score' in column:
        pass
    elif column not in numeric_columns:
        dummies.append(column)
            
def run_model_with_poly(dataframe, poly_n, modelo):
        
    poly = PolynomialFeatures(degree=poly_n)

    newX = dataframe[dummies+numeric_columns]
    print('newX shape', newX.shape)
    
    newY = dataframe['imdb_score']
    
    df_temp = pd.DataFrame(poly.fit_transform(newX[numeric_columns]))
    print('\ndf_temp shape', df_temp.shape)
    
    df_temp = pd.concat([df_temp,dataframe[dummies].reset_index()], axis=1)
    print('df_temp shape', df_temp.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(df_temp,newY,test_size=0.2)

    print('\ntrain shape', X_train.shape)
    print('teste shape', X_test.shape)
    
    modelo.fit(X_train, y_train)

    yhat = modelo.predict(X_train)
    yhat_test = modelo.predict(X_test[list(X_train)])

    print_metrics(y_train, yhat, y_test, yhat_test)
    return modelo

Comparando nossos dois modelos (MQO vs Arvores Aleatórias) parece que as segunda consegue superar as métricas no treino e no teste então vamos usar esse modelo em produção (ou seja, para realizar nosso trabalho).

In [29]:
model1 = run_model_with_poly(df, 1, LinearRegression())

newX shape (4937, 1046)

df_temp shape (4937, 15)
df_temp shape (4937, 1048)

train shape (3949, 1048)
teste shape (988, 1048)

-----Dados de Treino-----
MSE - treino 0.464623308253822
MAE - treino 0.3288167293337212
R2 - treino 0.6382568795489624

-----Dados de Teste-----
MSE - test 0.6392919158029365
MAE - test 0.49085686864009404
R2 - test 0.4585404877019176


In [30]:
model2 = run_model_with_poly(df, 2, LinearRegression())

newX shape (4937, 1046)

df_temp shape (4937, 120)
df_temp shape (4937, 1153)

train shape (3949, 1153)
teste shape (988, 1153)

-----Dados de Treino-----
MSE - treino 0.8402968139718888
MAE - treino 0.5330488904305888
R2 - treino 0.35109550966948877

-----Dados de Teste-----
MSE - test 1.0394061919562865
MAE - test 0.5225989807184295
R2 - test 0.08637961565854357


In [31]:
model3 = run_model_with_poly(df, 1, RandomForestRegressor())

newX shape (4937, 1046)

df_temp shape (4937, 15)
df_temp shape (4937, 1048)

train shape (3949, 1048)
teste shape (988, 1048)

-----Dados de Treino-----
MSE - treino 0.11800278551532034
MAE - treino 0.15999999999999925
R2 - treino 0.9067922373630145

-----Dados de Teste-----
MSE - test 0.6664698380566801
MAE - test 0.46000000000000085
R2 - test 0.4685376381152988


In [32]:
model4 = run_model_with_poly(df, 2, RandomForestRegressor())

newX shape (4937, 1046)

df_temp shape (4937, 120)
df_temp shape (4937, 1153)

train shape (3949, 1153)
teste shape (988, 1153)

-----Dados de Treino-----
MSE - treino 0.12479903773107116
MAE - treino 0.16999999999999904
R2 - treino 0.9002756187657337

-----Dados de Teste-----
MSE - test 0.6817317813765181
MAE - test 0.42500000000000027
R2 - test 0.4806721147829238


7) Após estarmos convictos que o modelo treinado tem uma boa relação entre o viés e variância (parametrizado pela complexidade do nosso modelo. Novamente, quanto mais complexo mais ele consegue capturar efeitos não previsto pelo modelo como comportamentos não lineares e interação das variáveis, mas tornando o modelo menos genérico, ou seja, diminuindo os scores dos dados de teste) vamos treinar o modelo final com os parâmetros que conseguiram os melhores resultados nos dados de teste.

In [58]:
# Dos nossos testes parece que a melhor combinação foi um RandomForestRegressor com polinomios = 1
# Então vamos fazer o treinamento do modelo final com TODOS os dados 
# (agora não podemos mais confiar nas métrica pois não teremos dados de teste)

Y = df['imdb_score']
X = df.drop(['imdb_score'], axis=1)

final_model = RandomForestRegressor().fit(X,Y)

In [59]:
len(list(X))

1046

# Passo 5 - Fazer Predições.

In [72]:
# Depois que temos o melhor modelo treinado vamos pegar os dados do mundo real e fazer previsão.

df2016_raw = pd.read_csv('test.csv')
df2016 = df2016_raw.copy(deep=True)
df2016.head()

# nesse nosso caso, temos o target nos dados "reais", geralmente não o teriamos, 
# aqui poderemos saber a perfomance do nosso algoritmo

,Unnamed: 0,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,10,Color,Zack Snyder,673.0,183.0,0.0,2000.0,Lauren Cohan,15000.0,330249062.0,...,3018.0,English,USA,PG-13,250000000.0,2016.0,4000.0,6.9,2.35,197000.0
1,27,Color,Anthony Russo,516.0,147.0,94.0,11000.0,Scarlett Johansson,21000.0,407197282.0,...,1022.0,English,USA,PG-13,250000000.0,2016.0,19000.0,8.2,2.35,72000.0
2,57,Color,Justin Lin,322.0,122.0,681.0,105.0,Melissa Roxburgh,998.0,130468626.0,...,432.0,English,USA,PG-13,185000000.0,2016.0,119.0,7.5,2.35,30000.0
3,63,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,...,239.0,English,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000.0
4,65,Color,Bryan Singer,396.0,144.0,0.0,1000.0,Michael Fassbender,34000.0,154985087.0,...,622.0,English,USA,PG-13,178000000.0,2016.0,13000.0,7.3,2.35,54000.0


In [61]:
# Todos os tratamentos de dados que fizemos no dataset original precisamos refazer aqui:

df2016 = df2016.drop(columns_to_drop, axis=1)

df2016[list(df_numeric)] = df2016[list(df_numeric)].fillna(df2016.mean())

df2016 = df2016.fillna('na')

df2016 = pd.get_dummies(df2016)

Y2016 = df2016['imdb_score']

In [62]:
len(list(X))

1046

In [63]:
len(list(df2016))

587

In [38]:
for i, dummy in enumerate(list(X)):
    if dummy not in list(df2016):
        df2016[dummy] = 0

print(f'dummies adicionadas:{i}')

NameError: name 'df2016' is not defined

In [65]:
X2016 = df2016[list(X)]

In [66]:
yhat = final_model.predict(X2016)

In [68]:
# Perfeito! Temos um array com os valores previstos
yhat

array([6.93, 7.41, 5.8 , 5.73, 6.03, 6.1 , 5.99, 4.16, 5.47, 5.94, 6.18,
       5.73, 3.1 , 6.54, 5.82, 6.17, 5.47, 5.85, 5.89, 5.71, 5.68, 5.01,
       5.9 , 5.31, 5.6 , 6.02, 6.79, 6.39, 7.76, 5.91, 5.92, 5.82, 5.93,
       6.12, 7.15, 5.43, 6.65, 5.93, 5.97, 6.37, 3.1 , 6.05, 4.93, 6.03,
       4.41, 5.99, 5.54, 6.71, 6.09, 6.71, 5.89, 6.44, 6.66, 5.65, 6.02,
       5.27, 6.29, 5.87, 5.48, 5.56, 5.88, 6.46, 4.92, 5.64, 5.64, 5.65,
       4.82, 5.39, 6.59, 6.03, 5.63, 6.42, 6.23, 6.37, 5.66, 5.45, 5.79,
       6.13, 4.45, 6.15, 5.29, 7.2 , 5.71, 6.77, 4.85, 5.84, 4.85, 5.39,
       5.21, 7.22, 5.56, 4.82, 5.88, 4.98, 6.2 , 5.91, 5.28, 5.21, 6.09,
       6.21, 6.05, 5.57, 5.08, 5.95, 5.97, 5.76])

In [77]:
# como podemos saber de qual filme se trata cada um dos score previstos?
# o algoritmo não diz qual é o nome do filme, mas a ordem em que passamos os dados se mantém ..
# então podemos adicionar uma coluna no nosso dataframe original com as predições!

df2016_raw['score_previsto'] = yhat
df2016_raw.head()

,Unnamed: 0,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,score_previsto
0,10,Color,Zack Snyder,673.0,183.0,0.0,2000.0,Lauren Cohan,15000.0,330249062.0,...,English,USA,PG-13,250000000.0,2016.0,4000.0,6.9,2.35,197000.0,6.93
1,27,Color,Anthony Russo,516.0,147.0,94.0,11000.0,Scarlett Johansson,21000.0,407197282.0,...,English,USA,PG-13,250000000.0,2016.0,19000.0,8.2,2.35,72000.0,7.41
2,57,Color,Justin Lin,322.0,122.0,681.0,105.0,Melissa Roxburgh,998.0,130468626.0,...,English,USA,PG-13,185000000.0,2016.0,119.0,7.5,2.35,30000.0,5.80
3,63,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,...,English,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000.0,5.73
4,65,Color,Bryan Singer,396.0,144.0,0.0,1000.0,Michael Fassbender,34000.0,154985087.0,...,English,USA,PG-13,178000000.0,2016.0,13000.0,7.3,2.35,54000.0,6.03


# Respondendo a pergunta do nosso Desafio:

In [92]:
result = df2016_raw[['movie_title','budget','score_previsto']]
result['ratio'] = (result['budget']/1000000)/(result['score_previsto'])

C:\Users\marcos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [93]:
# "piores" 10 filmes nas regras desse desafio, são os filmes que os scores custam mais caro.
result.sort_values(by=['ratio'], ascending=False).head(10)

,movie_title,budget,score_previsto,ratio
40,Ghostbusters,144000000.0,3.10,46.451613
12,Ghostbusters,144000000.0,3.10,46.451613
7,Independence Day: Resurgence,165000000.0,4.16,39.663462
0,Batman v Superman: Dawn of Justice,250000000.0,6.93,36.075036
1,Captain America: Civil War,250000000.0,7.41,33.738192
2,Star Trek Beyond,185000000.0,5.80,31.896552
11,The Legend of Tarzan,180000000.0,5.73,31.413613
3,The Legend of Tarzan,180000000.0,5.73,31.413613
4,X-Men: Apocalypse,178000000.0,6.03,29.519071
6,The Jungle Book,175000000.0,5.99,29.215359


In [39]:
# "melhores" 10 filmes nas regras desse desafio.
result.sort_values(by=['ratio'], ascending=True).head(10)

NameError: name 'result' is not defined

# Próximo passo, colocar o modelo em produção ...